In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import eat # Assuming 'eat.py' contains the EAT class implementation
import pandas as pd
from models.modelsFDH import FDH # Assuming 'models/modelsFDH.py' contains the FDH class implementation

# Set random seed for reproducibility if desired, otherwise remove or comment out
# np.random.seed(42)

# --- Constants based on the paper ---
# Changed TRIALS from 10 to 100 as per paper text (page 13)
TRIALS = 100
# Changed SAMPLE_SIZES to match paper tables (page 13, 15)
SAMPLE_SIZES = [50, 100, 150]

# --- Scenario Definitions based on Table 1 (page 13) ---
SCENARIOS = {
    1: {"inputs": 1,
        "outputs": 1,
        "func": lambda x: np.log(x[:,0]) + 3, # Use vectorized operations
        "inefficiency": lambda n: np.abs(np.random.normal(0, 0.4, size=n))},
    2: {"inputs": 2,
        "outputs": 1,
        # Use vectorized operations
        "func": lambda x: 0.1*x[:,0] + 0.1*x[:,1] + 0.3*((x[:,0] * x[:,1])**(1/2)),
        "inefficiency": lambda n: np.abs(np.random.normal(0, 0.4, size=n))},
    3: {"inputs": 3,
        "outputs": 1,
        # Use vectorized operations
        "func": lambda x: 0.1*x[:,0] + 0.1*x[:,1] + 0.1*x[:,2] + 0.3*(x[:,0]*x[:,1]*x[:,2])**(1/3),
        "inefficiency": lambda n: np.abs(np.random.normal(0, 0.4, size=n))},
    4: {"inputs": 9,
        "outputs": 1,
        # Use vectorized operations - Paper function f(x)=Π(xi^0.1) is ambiguous, using Lee and Cai (2020) interpretation if available or assuming geometric mean variant.
        # This assumes f(x) = (Πxi)^(1/9) -> exp( (1/9) * sum(log(xi)) ). Let's use the Cobb-Douglas form Π(x_i ^ (1/9))
        "func": lambda x: np.prod(x**(1/9), axis=1), # Use x directly as it's (n, 9)
        "inefficiency": lambda n: np.abs(np.random.normal(0, 0.4, size=n))},
    5: { # Multi-output scenario based on Perelman and Santín (2009) referenced in paper
        "inputs": 2,
        "outputs": 2,
        # The function definition here describes the *relationship*, not a direct generation formula for y.
        # Generation is handled separately in generate_data based on the paper's description/likely source method.
        "func": None, # Mark as None, generation logic is specific
        # Inefficiency applied component-wise for multi-output
        "inefficiency": lambda n: np.abs(np.random.normal(0, 0.4, size=(n, 2))),
    },
}

# --- Data Generation Function ---
def generate_data(scenario_id, scenario, n):
    """Generates data for a given scenario and sample size."""
    x = np.random.uniform(1, 10, size=(n, scenario["inputs"]))
    true_frontier_y = np.zeros((n, scenario["outputs"])) # Initialize true frontier output array

    # --- Calculate True Frontier Output (f(x)) ---
    if scenario_id == 5:
        # Multi-output generation needs specific logic (e.g., solving the implicit function)
        # Replicating Perelman and Santín (2009) method precisely might be complex.
        # Using a simplified placeholder or assumed generation logic if the exact method isn't implemented.
        # Placeholder: Generate y1, y2 based on some function of x, ensuring positivity and reasonable scale.
        # This part *must* be correctly implemented based on the paper's source method for Scenario 5 results to be valid.
        # Example placeholder (likely incorrect w.r.t paper's source):
        true_frontier_y[:, 0] = 0.5 * x[:, 0] + 0.3 * x[:, 1] + 2
        true_frontier_y[:, 1] = 0.2 * x[:, 0] + 0.6 * x[:, 1] + 1
        # --- !!! Crucial: Replace placeholder with correct Scenario 5 generation logic !!! ---

    else:
        # Single output scenarios (1-4)
        true_frontier_y = scenario["func"](x)
        # Reshape to (n, 1) if it's not already
        if true_frontier_y.ndim == 1:
             true_frontier_y = true_frontier_y.reshape(-1, 1)

    # --- Generate Inefficiency Term (u) ---
    inefficiency = scenario["inefficiency"](n)
    # Ensure inefficiency term has the same shape as output for subtraction
    if inefficiency.ndim == 1 and scenario["outputs"] > 1:
         # This case shouldn't happen with the current SCENARIOS definition but added for robustness
         inefficiency = np.tile(inefficiency.reshape(-1, 1), (1, scenario["outputs"]))
    elif inefficiency.ndim == 1 and scenario["outputs"] == 1:
         inefficiency = inefficiency.reshape(-1, 1)


    # --- Calculate Observed Output (y = f(x) - u) ---
    observed_y = true_frontier_y - inefficiency

    # Ensure observed outputs are positive (common practice in production frontiers)
    # This step wasn't explicitly mentioned for scenarios 1-4 but often necessary.
    # Adjust if the paper implies negative outputs are possible.
    observed_y[observed_y < 0] = 1e-6 # Replace negative values with a small positive number

    # --- Return true frontier values for metric calculation ---
    # Changed return value to include true_frontier_y
    return x, observed_y, true_frontier_y

# --- Monte Carlo Simulation Function ---
def monte_carlo_testing():
    """Runs the Monte Carlo simulation."""
    results = {}
    discrepancy_counts = {} # To store counts for FDH vs Deep EAT discrepancy (optional, like in paper)

    for scenario_id, scenario in SCENARIOS.items():
        print(f"--- Running Scenario {scenario_id} ---")
        discrepancy_counts[scenario_id] = {}
        for n in SAMPLE_SIZES:
            print(f"  Sample Size: {n}")
            mse_fdh_list, mse_eat_list = [], []
            bias_fdh_list, bias_eat_list = [], []
            abs_bias_fdh_list, abs_bias_eat_list = [], []
            discrepancies = 0 # Counter for FDH != Deep EAT

            for trial in range(TRIALS):
                if (trial + 1) % 20 == 0: # Print progress
                   print(f"    Trial {trial + 1}/{TRIALS}")

                # Generate data, now including true_frontier_y
                x, y_observed, y_true_frontier = generate_data(scenario_id, scenario, n)

                # --- FDH Estimation ---
                # Ensure y_observed is 2D
                if y_observed.ndim == 1:
                    y_observed_fdh = y_observed.reshape(-1, 1)
                else:
                    y_observed_fdh = y_observed

                # Assuming FDH class takes x, y and calculates projections
                # Note: FDH itself doesn't estimate f(x) directly, it finds efficient peers.
                # The 'estimated frontier' for FDH at point xi is the maximum output achievable
                # by units dominating xi. For comparison, we often use the radial projection.
                try:
                    fdh_model = FDH(x, y_observed_fdh)
                    # Assuming fdh_output_vrs returns efficiency scores or projections
                    # If it returns efficiency scores 'eff': y_fdh_est = y_observed / eff (output oriented)
                    # If it returns projected points: y_fdh_est = projected_y
                    # Let's assume it gives projected points 'y_proj' as a common output format
                    # Need to confirm what `fdh_output_vrs` actually returns
                    # Placeholder: Assuming it returns a dataframe with projections or scores
                    # Using radial projection calculation here as it's common
                    # This part *requires* knowing the exact output of your FDH implementation
                    # Example: If it returns efficiency scores:
                    # fdh_results_df = fdh_model.fdh_output_vrs() # Assuming returns DataFrame
                    # efficiency_scores = fdh_results_df["efficiency"].values
                    # # Handle potential division by zero or very small efficiency scores if any
                    # efficiency_scores[efficiency_scores < 1e-9] = 1e-9
                    # # Calculate output-oriented projection
                    # y_fdh_est = y_observed_fdh * efficiency_scores.reshape(-1, 1)
                    
                    # Using original code's structure assuming it calculates projection correctly
                    fdh_results_df = fdh_model.fdh_output_vrs()
                    y_fdh_est = y_observed_fdh * fdh_results_df["efficiency"].values.reshape(-1, 1)


                except Exception as e:
                    print(f"Error in FDH Trial {trial+1} (Scenario {scenario_id}, n={n}): {e}")
                    # Skip trial or handle error appropriately
                    y_fdh_est = np.full_like(y_true_frontier, np.nan) # Fill with NaN on error


                # --- EAT Estimation ---
                # Prepare DataFrame for EAT class
                x_cols = [f"x[{i}]" for i in range(scenario["inputs"])]
                y_cols = [f"y[{i}]" for i in range(scenario["outputs"])]
                df = pd.DataFrame(x, columns=x_cols)
                df[y_cols] = y_observed # Use observed y for training EAT

                try:
                    # Using numStop=5 as suggested by Breiman et al. (1984) and implicit in paper (page 6)
                    # Using fold=10 (common default for cross-validation) if 5 isn't standard in your lib
                    eat_model = eat.EAT(
                        df, x_cols, y_cols,
                        numStop=5, # Changed numStop based on paper discussion
                        fold=10) # Standard CV folds, paper doesn't specify
                    eat_model.fit() # Fit includes pruning

                    # Predict using the same x values
                    data_pred = df.loc[:, x_cols]
                    y_eat_pred_df = eat_model.predict(data_pred) # Assuming predict takes df and returns df

                    # Extract predicted columns (assuming names like 'p_y[0]', 'p_y[1]', ...)
                    pred_y_cols = [f"p_y[{i}]" for i in range(scenario["outputs"])]
                    y_eat_est = y_eat_pred_df[pred_y_cols].values

                except Exception as e:
                    print(f"Error in EAT Trial {trial+1} (Scenario {scenario_id}, n={n}): {e}")
                    y_eat_est = np.full_like(y_true_frontier, np.nan) # Fill with NaN on error

                # --- Calculate Metrics ---
                # Pass the TRUE frontier values for comparison
                mse_fdh, bias_fdh, abs_bias_fdh = calculate_metrics(y_true_frontier, y_fdh_est)
                mse_fdh_list.append(mse_fdh)
                bias_fdh_list.append(bias_fdh)
                abs_bias_fdh_list.append(abs_bias_fdh)

                mse_eat, bias_eat, abs_bias_eat = calculate_metrics(y_true_frontier, y_eat_est)
                mse_eat_list.append(mse_eat)
                bias_eat_list.append(bias_eat)
                abs_bias_eat_list.append(abs_bias_eat)


            # --- Store Average Results for (Scenario, n) ---
            # Added checks for NaN in case of errors during trials
            results[(scenario_id, n)] = {
                "FDH_MSE": np.nanmean(mse_fdh_list),
                "EAT_MSE": np.nanmean(mse_eat_list),
                "FDH_Bias": np.nanmean(bias_fdh_list),
                "EAT_Bias": np.nanmean(bias_eat_list),
                "FDH_AbsBias": np.nanmean(abs_bias_fdh_list),
                "EAT_AbsBias": np.nanmean(abs_bias_eat_list),
            }
            # Optional: Calculate discrepancy percentage like in paper's Table 2
            # discrepancy_percentage = (discrepancies / (n * TRIALS)) * 100
            # results[(scenario_id, n)]["Discrepancy_Percent"] = discrepancy_percentage

    return results

# --- Metrics Calculation Function ---
def calculate_metrics(true_values, estimates):
    """Calculates MSE, Bias, and Absolute Bias."""
    # Ensure inputs are numpy arrays
    true_values = np.asarray(true_values)
    estimates = np.asarray(estimates)

    # Handle potential NaNs from failed trials
    mask = ~np.isnan(estimates) & ~np.isnan(true_values)
    if np.sum(mask) == 0: # All estimates failed
        return np.nan, np.nan, np.nan

    valid_true = true_values[mask]
    valid_est = estimates[mask]

    # Calculate metrics only on valid pairs
    mse = np.mean((valid_est - valid_true) ** 2)
    bias = np.mean(valid_est - valid_true)
    abs_bias = np.mean(np.abs(valid_est - valid_true))

    return mse, bias, abs_bias

# --- Run Simulation and Print Results ---
results = monte_carlo_testing()

print("\n--- Monte Carlo Simulation Results ---")
# Prepare data for table display
table_data = []
for (scenario_id, n), metrics in results.items():
    # Calculate percentage improvement for EAT over FDH (Reduction)
    mse_reduction = ((metrics['FDH_MSE'] - metrics['EAT_MSE']) / metrics['FDH_MSE']) * 100 if metrics['FDH_MSE'] else 0
    # Bias reduction - tricky sign convention, use absolute bias reduction
    abs_bias_reduction = ((metrics['FDH_AbsBias'] - metrics['EAT_AbsBias']) / metrics['FDH_AbsBias']) * 100 if metrics['FDH_AbsBias'] else 0

    table_data.append({
        "Scenario": scenario_id,
        "N": n,
        "FDH MSE": metrics['FDH_MSE'],
        "EAT MSE": metrics['EAT_MSE'],
        "MSE Reduction (%)": mse_reduction,
        "FDH Bias": metrics['FDH_Bias'],
        "EAT Bias": metrics['EAT_Bias'],
        "FDH AbsBias": metrics['FDH_AbsBias'],
        "EAT AbsBias": metrics['EAT_AbsBias'],
        "AbsBias Reduction (%)": abs_bias_reduction
        # Add "Discrepancy (%)": metrics.get("Discrepancy_Percent", np.nan) if calculated
    })

# Display results in a pandas DataFrame
results_df = pd.DataFrame(table_data)
pd.set_option('display.float_format', lambda x: '%.4f' % x) # Format floats
print(results_df.to_string(index=False))

# Optional: Save results to CSV
# results_df.to_csv("monte_carlo_results.csv", index=False)